In [4]:
tuple = ('left', 44.44, "temp", 99.99)

In [7]:
tuple[1]

44.44

In [8]:
txt = "   wow"

In [9]:
txt

'   wow'

In [11]:
txt.strip()

'wow'

In [12]:
print('{0:-2%}'.format(1.0/3))

33.333333%


In [14]:
print('abcd'.replace('cd','12'))

ab12


In [15]:
my_string = 'whais'
for i in range(len(my_string)):
    print(i)

0
1
2
3
4


In [16]:
print(3>=3)

True


In [17]:
N ='3.0' 

In [18]:
type(N)

str